In [25]:
import os
import pandas as pd
import json


In [28]:
results_root = "./results/all"

# Get all model folders
folders = [f for f in os.listdir(results_root) if os.path.isdir(os.path.join(results_root, f))]
all_results = []

# Process each folder
for folder in folders:
    json_path = os.path.join(results_root, folder, 'all_results.json')
    if not os.path.exists(json_path):
        print(f"Skipping {folder}, no results {json_path} file found.")
        continue

    with open(json_path, 'r') as f:
        data = json.load(f)
        groups = data['groups']
        print(json_path)
        print(groups)
        # Extract results
        df = pd.DataFrame(
            {
                "model_name": folder,
                "blimp_supplement_acc": groups["blimp_supplement"]["acc,none"],
                "blimp_supplement_stderr": groups["blimp_supplement"]["acc_stderr,none"],
                "blimp_filtered_acc": groups["blimp_filtered"]["acc,none"],
                "blimp_filtered_stderr": groups["blimp_filtered"]["acc_stderr,none"],
                "ewok_filtered_acc": groups["ewok_filtered"]["acc,none"],
                "ewok_filtered_stderr": groups["ewok_filtered"]["acc_stderr,none"],
            },
            index=[0],
        )
        all_results.append(df)

# Combine all results
combined_results = pd.concat(all_results, ignore_index=True)
combined_results = combined_results.sort_values(by='model_name').reset_index(drop=True)
combined_results

./results/all/PalenkaLlama1-58M-strict-L0.1-H0.3_20250419_192616/all_results.json
{'ewok_filtered': {'acc,none': 0.498806338617783, 'acc_stderr,none': 0.005731827417496267, 'alias': 'ewok_filtered'}, 'blimp_supplement': {'acc,none': 0.5990337302428236, 'acc_stderr,none': 0.005262043444131852, 'alias': 'blimp_supplement'}, 'blimp_filtered': {'acc,none': 0.6933993827879917, 'acc_stderr,none': 0.0016156697990275413, 'alias': 'blimp_filtered'}}
./results/all/PalenkaLlama1-58M-strict-L0.1-H0.5_20250419_215705/all_results.json
{'ewok_filtered': {'acc,none': 0.502875831389563, 'acc_stderr,none': 0.005731205587920797, 'alias': 'ewok_filtered'}, 'blimp_supplement': {'acc,none': 0.6061930727592519, 'acc_stderr,none': 0.004981778196749111, 'alias': 'blimp_supplement'}, 'blimp_filtered': {'acc,none': 0.6941923683085492, 'acc_stderr,none': 0.001621555999646299, 'alias': 'blimp_filtered'}}
./results/all/BabyLlama1-58M-strict/all_results.json
{'ewok_filtered': {'acc,none': 0.5005777851120263, 'acc_st

,model_name,blimp_supplement_acc,blimp_supplement_stderr,blimp_filtered_acc,blimp_filtered_stderr,ewok_filtered_acc,ewok_filtered_stderr
0,BabyLlama1-58M-strict,0.581199,0.005364,0.676288,0.001620,0.500578,0.005731
1,DistilledGPT-44M-strict,0.588133,0.005255,0.657983,0.001628,0.499610,0.005732
2,GPT2-44M-strict,0.591463,0.005589,0.633241,0.001716,0.501399,0.005732
3,GPT2-705M-strict,0.574093,0.005795,0.657127,0.001667,0.500947,0.005731
4,GPT2-small-97M-strict,0.562790,0.005828,0.662344,0.001667,0.505874,0.005730
5,Llama-360M-strict,0.609625,0.005478,0.653927,0.001681,0.499416,0.005731
6,Llama-60M-strict,0.566840,0.005669,0.637332,0.001741,0.498886,0.005731
7,PalenkaLlama1-58M-strict-L0.1-H0.1_20250419_16...,0.596101,0.005435,0.693733,0.001617,0.499761,0.005731
8,PalenkaLlama1-58M-strict-L0.1-H0.3_20250419_19...,0.599034,0.005262,0.693399,0.001616,0.498806,0.005732
9,PalenkaLlama1-58M-strict-L0.1-H0.5_20250419_21...,0.606193,0.004982,0.694192,0.001622,0.502876,0.005731


In [ ]:
# Define a helper to format "<acc> ± <stderr>"
def fmt(score, stderr, score_rounding=3, stderr_rounding=4):
    return f"{score:.{score_rounding}f} ± {stderr:.{stderr_rounding}f}"

# Apply formatting
combined_results['blimp_supplement'] = combined_results.apply(
    lambda row: fmt(row['blimp_supplement_acc'], row['blimp_supplement_stderr']), axis=1
)
combined_results['blimp_filtered'] = combined_results.apply(
    lambda row: fmt(row['blimp_filtered_acc'], row['blimp_filtered_stderr']), axis=1
)
combined_results['ewok_filtered'] = combined_results.apply(
    lambda row: fmt(row['ewok_filtered_acc'], row['ewok_filtered_stderr']), axis=1
)

# Select only the formatted columns
final_df = combined_results[['model_name', 'blimp_supplement', 'blimp_filtered', 'ewok_filtered']]

# Print as a Markdown table
print(final_df.to_markdown(index=True))

|    | model_name                                         | blimp_supplement   | blimp_filtered   | ewok_filtered   |
|---:|:---------------------------------------------------|:-------------------|:-----------------|:----------------|
|  0 | BabyLlama1-58M-strict                              | 0.581 ± 0.0054     | 0.676 ± 0.0016   | 0.501 ± 0.0057  |
|  1 | DistilledGPT-44M-strict                            | 0.588 ± 0.0053     | 0.658 ± 0.0016   | 0.500 ± 0.0057  |
|  2 | GPT2-44M-strict                                    | 0.591 ± 0.0056     | 0.633 ± 0.0017   | 0.501 ± 0.0057  |
|  3 | GPT2-705M-strict                                   | 0.574 ± 0.0058     | 0.657 ± 0.0017   | 0.501 ± 0.0057  |
|  4 | GPT2-small-97M-strict                              | 0.563 ± 0.0058     | 0.662 ± 0.0017   | 0.506 ± 0.0057  |
|  5 | Llama-360M-strict                                  | 0.610 ± 0.0055     | 0.654 ± 0.0017   | 0.499 ± 0.0057  |
|  6 | Llama-60M-strict                                 